# ENTREGABLE 3

# INSTRUCCIONES

Utilizar el archivo CSV (dataset_default.csv).

# ¿POR QUÉ Y CUANDO DEBO ESCALAR LOS DATOS?

En este notebook vamos a analizar el efecto que el escalamiento de los datos tiene en el desempeño de diferentes modelos de *Machine Learning*.

Y con los resultados que obtengamos tendremos claro cuando resulta conveniente escalar los datos y cuando no.

## 1. El problema a resolver

Usaremos un set de datos que contiene la información asociada al pago ("no default") e impago ("default") de un crédito bancario para un total de 5.383 personas.

Construiremos varios modelos de *Machine Learning* que tomarán como entrada datos como el puntaje crediticio, los ingresos mensuales y el monto del préstamo, y aprenda a predecir si el usuario pagará a tiempo ("no default", 0) o se retrasará en sus pagos ("default", 1).

Y cada modelo será entrenado con datos con y sin escalar y veremos el efecto que esto tiene en cada modelo.

In [ ]:
# Importar las librerías
import pandas as pd
import plotly.express as px
from google.colab import drive

# Montar Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


## 2. El set de datos

In [ ]:
# Leer el dataset
RUTA = '/content/dataset_default.csv'
df = pd.read_csv(RUTA)
df

,Puntaje crediticio,Ingresos mensuales (USD),Monto del préstamo (USD),¿Default/No default?
0,5.3,6074.6,34159.9,0
1,4.1,4955.0,35168.6,0
2,4.4,4573.6,26852.8,0
3,3.1,6019.0,31500.9,0
4,5.0,1481.1,38094.6,1
...,...,...,...,...
5378,2.9,2558.5,47140.9,1
5379,0.7,3594.2,46654.5,1
5380,6.4,681.2,44423.2,1
5381,2.5,5172.0,24832.9,0


Veamos un conteo de categorías para verificar si el set de datos está balanceado:

In [ ]:
df['¿Default/No default?'].value_counts(normalize=True)

¿Default/No default?
1    0.507152
0    0.492848
Name: proportion, dtype: float64

Veamos los rangos de valores que alcanzan los datos que usaremos como entrada de cada modelo:

In [ ]:
for col in df.columns[0:3]:
    # Calcular valores mínimo y máximo de la columna
    minimo = df[col].min()
    maximo = df[col].max()
    print(f'Máximo/mínimo columna "{col}": {maximo:.1f}/{minimo:.1f}')

Máximo/mínimo columna "Puntaje crediticio": 7.5/0.0
Máximo/mínimo columna "Ingresos mensuales (USD)": 6997.8/1.4
Máximo/mínimo columna "Monto del préstamo (USD)": 49991.1/20023.4


Y vemos que **cada columna tiene una escala diferente**.

Y por último hagamos un gráfico 3D de los datos para entender sus principales características:

In [ ]:
fig = px.scatter_3d(df, x='Puntaje crediticio', y='Ingresos mensuales (USD)',
                    z='Monto del préstamo (USD)', color='¿Default/No default?',
                    size_max=6, opacity = 0.5)
fig.show()

##3. ¿Qué es el escalamiento de los datos?

> El escalamiento consiste en hacer que cada variable(columna) tenga aproximadamente **el mismo rango de valores**.

Por ejemplo, si antes del escalamiento tenemos:

- `Puntaje crediticio`: [0.0 - 7.5]
- `Ingresos mensuales`: [1.4 - 6997.8]
- `Monto del préstamo`: [20023.4 - 49991.1]

Después del escalamiento podríamos tener:

- Todas las variables entre 0 y 1 (normalización)
- Todas las variables con valores promedio de 0 y desviaciones estándar de 1 (estandarización)

## 4. Funciones requeridas

Para introducir los datos a cada modelo necesitaremos:

1. obtener las entradas (`X`) y salida (`Y`) del modelo a partir del set de datos original
2. crear los sets de entrenamiento, validación y prueba a partir de `X` y `Y`

Además, para entrenar y validar cada modelo con los datos escalados necesitaremos crear una función para ESCALAR los datos.


In [ ]:
# Obtén los vectores X(características) e Y(target)
## A COMPLETAR

# Importar la librería pandas
import pandas as pd

# Suponiendo que 'df' es tu DataFrame con los datos
X = df.drop(columns=['¿Default/No default?'])  # Características (todas las columnas excepto la de default)
Y = df['¿Default/No default?']  # Variable objetivo (columna de default)

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir datos en entrenamiento y prueba (85% para entrenamiento y 15% para prueba)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

# Dividir datos de entrenamiento en entrenamiento y validación (70% para entrenamiento y 15% para validación)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1765, random_state=42)  # 0.15 / (1 - 0.15)

# Verificar las formas de los conjuntos
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)


(3767, 3) (3767,)
(808, 3) (808,)
(808, 3) (808,)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Haremos escalamiento entre 0 y 1 (normalización)

def escalar_datos(xtr, xvl, xts):
    '''Escalar datos de entrada (entrenamiento, validación y prueba)
    en el rango de 0 a 1'''
    scaler = MinMaxScaler()

    # Fit + transform sobre el set de entrenamiento
    xtr_s = scaler.fit_transform(xtr)

    # Transform sobre prueba y validación
    xvl_s = scaler.transform(xvl)
    xts_s = scaler.transform(xts)

    return xtr_s, xvl_s, xts_s

# Ejecutar la función
x_train_s, x_val_s, x_test_s = escalar_datos(x_train, x_val, x_test)

# Verificar
print(x_train_s.min(axis=0),x_train_s.max(axis=0))
print(x_val_s.min(axis=0),x_val_s.max(axis=0))
print(x_test_s.min(axis=0),x_test_s.max(axis=0))

[0. 0. 0.] [1. 1. 1.]
[0.         0.00071465 0.00060065] [1.         0.99995712 0.99845834]
[0.         0.00663198 0.00173186] [1.         1.         0.99981313]


Ya tenemos las funciones básicas requeridas. Ahora sí veamos el efecto que tiene el escalamiento en diferentes tipos de modelos.

##5. Efecto del escalamiento en diferentes modelos de *Machine Learning*

En este experimento probaremos con dos tipos de modelos:

1. Un árbol de clasificación, que es "inmune" al escalamiento
2. Un clasificador kNN (*k-nearest neighbors*) que es sensible al escalamiento
3. SVM

Veamos el efecto que el escalamiento tiene sobre estos dos tipos de modelos:

###5.1. Árboles de clasificación

> En los árboles de clasificación (y los árboles de regresión y los bosques aleatorios) **el desempeño es el mismo con o sin escalamiento de los datos**.

Lo anterior se debe a que al momento de clasificar (o predecir un dato numérico) internamente los árboles/bosques calculan umbrales que están dados en la escala en la que se encuentren los datos:

![](https://drive.google.com/uc?export=view&id=1FhtonygNk7DfSoVooMQQGjoQGYQlXqJh
)

Como el umbral se desplaza en la misma proporción que lo hace el escalamiento, todos estos modelos tendrán el mismo desempeño.

Verifiquemos esto con nuestro set de datos.

Creemos una función para entrenar y evaluar un sencillo árbol de clasificación:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def entrenar_evaluar_arbol(x_train, y_train, x_val, y_val):
    # Crear y entrenar el árbol de clasificación sin escalar los datos
    tree_clf = DecisionTreeClassifier(random_state=42)
    tree_clf.fit(x_train, y_train)

    # Predecir en el conjunto de validación sin escalar
    y_pred = tree_clf.predict(x_val)

    # Calcular la precisión sin escalar
    accuracy = accuracy_score(y_val, y_pred)
    print("Precisión sin escalar:", accuracy)

    # Crear y entrenar el árbol de clasificación con datos escalados
    tree_clf_scaled = DecisionTreeClassifier(random_state=42)
    tree_clf_scaled.fit(x_train_s, y_train)

    # Predecir en el conjunto de validación con datos escalados
    y_pred_scaled = tree_clf_scaled.predict(x_val_s)

    # Calcular la precisión con datos escalados
    accuracy_scaled = accuracy_score(y_val, y_pred_scaled)
    print("Precisión con escalamiento:", accuracy_scaled)

# Llamar a la función con los conjuntos de datos escalados y sin escalar
entrenar_evaluar_arbol(x_train, y_train, x_val, y_val)


Precisión sin escalar: 0.9764851485148515
Precisión con escalamiento: 0.9764851485148515



Dado que la precisión del árbol de clasificación en el conjunto de validación es idéntica tanto con los datos sin escalar como con los datos escalados, podemos deducir que en este caso específico el escalado de datos no tuvo ningún impacto en el desempeño del modelo. Esto confirma lo que mencionamos anteriormente: los árboles de clasificación (y los árboles de regresión y los bosques aleatorios) son "inmunes" al escalamiento de los datos.

###5.2. Clasificador kNN (*k-nearest neighbors*)

> En los clasificadores kNN **el desempeño NO necesariamente será el mismo con o sin escalamiento de los datos**.

Este clasificador toma el dato a clasificar y calcula su distancia con respecto a la totalidad de los datos. Luego determina la categoría con base en las categorías de los *k* datos más cercanos (con la menor distancia).

La clave es la forma como se calculan las distancias entre el punto a clasificar y los demás puntos con categorías conocidas:

![](https://drive.google.com/uc?export=view&id=1FkAVal4XEBb6HcpE4IS6yL5N7OEsCuaJ
)

Si cada característica tiene una escala diferente entonces aquellas características con mayores escalas tendrán un mayor efecto en el valor de la distancia obtenida.

Veamos nuevamente este comportamiento para el set de datos que estamos usando.

Creemos una función para entrenar y evaluar un clasificador kNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def entrenar_evaluar_knn(x_train, y_train, x_val, y_val):
    # Crear y entrenar el clasificador kNN sin escalar los datos
    knn_clf = KNeighborsClassifier()
    knn_clf.fit(x_train, y_train)

    # Predecir en el conjunto de validación sin escalar
    y_pred = knn_clf.predict(x_val)

    # Calcular la precisión sin escalar
    accuracy = accuracy_score(y_val, y_pred)
    print("Precisión sin escalar:", accuracy)

    # Crear y entrenar el clasificador kNN con datos escalados
    knn_clf_scaled = KNeighborsClassifier()
    knn_clf_scaled.fit(x_train_s, y_train)

    # Predecir en el conjunto de validación con datos escalados
    y_pred_scaled = knn_clf_scaled.predict(x_val_s)

    # Calcular la precisión con datos escalados
    accuracy_scaled = accuracy_score(y_val, y_pred_scaled)
    print("Precisión con escalamiento:", accuracy_scaled)

# Llamar a la función con los conjuntos de datos escalados y sin escalar
entrenar_evaluar_knn(x_train, y_train, x_val, y_val)


Precisión sin escalar: 0.9777227722772277
Precisión con escalamiento: 0.9727722772277227



Dado que la precisión del clasificador kNN en el conjunto de validación es ligeramente menor cuando se utilizan datos escalados en comparación con los datos sin escalar, podemos deducir que en este caso específico el escalamiento de datos ha tenido un impacto negativo en el desempeño del modelo.

En resumen, para este caso específico, el escalamiento de datos no es beneficioso y puede incluso reducir ligeramente la precisión del modelo kNN.


###5.3. Clasificador SVM (*Support Vector Machine*)

Comprobar ahora como afecta la normalización usando un modelo de SVM

In [ ]:
from sklearn.svm import SVC

def entrenar_evaluar_svm(x_train, y_train, x_val, y_val):
    # Crear y entrenar el clasificador SVM sin escalar los datos
    svm_clf = SVC()
    svm_clf.fit(x_train, y_train)

    # Predecir en el conjunto de validación sin escalar
    y_pred = svm_clf.predict(x_val)

    # Calcular la precisión sin escalar
    accuracy = accuracy_score(y_val, y_pred)
    print("Precisión sin escalar:", accuracy)

    # Crear y entrenar el clasificador SVM con datos escalados
    svm_clf_scaled = SVC()
    svm_clf_scaled.fit(x_train_s, y_train)

    # Predecir en el conjunto de validación con datos escalados
    y_pred_scaled = svm_clf_scaled.predict(x_val_s)

    # Calcular la precisión con datos escalados
    accuracy_scaled = accuracy_score(y_val, y_pred_scaled)
    print("Precisión con escalamiento:", accuracy_scaled)

# Llamar a la función con los conjuntos de datos escalados y sin escalar
entrenar_evaluar_svm(x_train, y_train, x_val, y_val)


Precisión sin escalar: 0.9628712871287128
Precisión con escalamiento: 0.9727722772277227


Dado que la precisión del clasificador SVM en el conjunto de validación es ligeramente mayor cuando se utilizan datos escalados en comparación con los datos sin escalar, podemos deducir que en este caso específico el escalamiento de datos ha tenido un impacto positivo en el desempeño del modelo.

##6. Conclusiones

1. **Árboles de Clasificación**: Los árboles de clasificación (y modelos relacionados como los árboles de regresión y los bosques aleatorios) son inmunes al escalamiento de datos. Esto significa que no se observa ninguna mejora en el desempeño al escalar los datos para este tipo de modelo.

2. **Clasificador kNN**: El desempeño del clasificador kNN puede variar con el escalamiento de datos. En el caso específico analizado, el escalamiento de datos no proporcionó ninguna mejora y, de hecho, se observó una ligera disminución en la precisión del modelo cuando se aplicó el escalamiento.

3. **Clasificador SVM**: El escalamiento de datos tuvo un impacto positivo en el desempeño del clasificador SVM en el caso analizado. Se observó una ligera mejora en la precisión del modelo cuando se utilizaron datos escalados en comparación con datos sin escalar.